# Swaps
- GetSwaps uses musd-pools-mezo


In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from mezo.currency_utils import format_musd_currency_columns, get_token_prices
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import SubgraphClient

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [ ]:
SWAPS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_SWAPS = """
query GetSwaps($skip: Int!) {
  swaps(
  first: 1000
  orderBy: timestamp_
  orderDirection: desc
  skip: $skip
  ) {
    timestamp_
    sender
    to
    contractId_
    amount0In
    amount0Out
    amount1In
    amount1Out
    transactionHash_
    block_number
  }
}
"""

In [ ]:
# Instantiate subgraph
swaps = SubgraphClient(
    url = SWAPS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

In [ ]:
# Fetch subgraph data
swap_data =  swaps.fetch_subgraph_data(GET_SWAPS, 'swaps')

In [11]:
# Save vaults data df

swaps_df = pd.DataFrame(swap_data)
display(swaps_df)

,timestamp_,sender,to,contractId_,amount0In,amount0Out,amount1In,amount1Out,transactionHash_,block_number
0,1755884578,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,0,16995370113873,1990000000000000000,0,0x5337d050a1082d36701ecf7a7ceaf108f37a86a67432...,2625985
1,1755884524,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x1ddd75b7cd221cca5b39f4dd88c2d54737357361,0x52e604c44417233b6ccedddc0d640a405caacefb,0,42362560058057,4960000000000000000,0,0xb5ca78eec6d0792d7dc74332d7ff9f9cfd8e0014302a...,2625970
2,1755884466,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,8583984375000,0,0,999000331300290689,0xcfdd976fbfc24f9c8a3ec978ae24d188ebfd75d2f6d5...,2625954
3,1755884437,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x99b340d3ee7b73de63f12f8b6ee667ea2d055ccd,0x52e604c44417233b6ccedddc0d640a405caacefb,0,21523810556733,2520000000000000000,0,0x6af958e749bd4b63131d28fce0125c3b1566afb668ee...,2625946
4,1755884437,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xd39abf814ba9bd8e7b64cb6505870b636d695d8f,0x52e604c44417233b6ccedddc0d640a405caacefb,0,21524636174687,2520000000000000000,0,0x12aa4abb6956d3eb42744f722f53772446374ff7dab2...,2625946
...,...,...,...,...,...,...,...,...,...,...
32108,1752013895,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,250000000000000,0,0,28463880024065974068,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293
32109,1751924558,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x52e604c44417233b6ccedddc0d640a405caacefb,0,157198903110751,18910000000000000000,0,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335
32110,1751924405,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,2010400,0,0,2009989838895249558,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292
32111,1750952336,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x10906a9e9215939561597b4c8e4b98f93c02031a,0,999896001631977073,1000000,0,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305


In [12]:

POOLS_MAP = {
    '0x52e604c44417233b6ccedddc0d640a405caacefb': 'musd_btc_pool',
    '0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356': 'musd_musdc_pool',
    '0x10906a9e9215939561597b4c8e4b98f93c02031a': 'musd_musdt_pool'
}

In [13]:
swaps_df['pool'] = swaps_df['contractId_'].map(POOLS_MAP)

In [15]:
format_datetimes(swaps_df, date_columns=['timestamp_'])

,timestamp_,sender,to,contractId_,amount0In,amount0Out,amount1In,amount1Out,transactionHash_,block_number,pool
0,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,0,16995370113873,1990000000000000000,0,0x5337d050a1082d36701ecf7a7ceaf108f37a86a67432...,2625985,musd_btc_pool
1,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x1ddd75b7cd221cca5b39f4dd88c2d54737357361,0x52e604c44417233b6ccedddc0d640a405caacefb,0,42362560058057,4960000000000000000,0,0xb5ca78eec6d0792d7dc74332d7ff9f9cfd8e0014302a...,2625970,musd_btc_pool
2,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,8583984375000,0,0,999000331300290689,0xcfdd976fbfc24f9c8a3ec978ae24d188ebfd75d2f6d5...,2625954,musd_btc_pool
3,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x99b340d3ee7b73de63f12f8b6ee667ea2d055ccd,0x52e604c44417233b6ccedddc0d640a405caacefb,0,21523810556733,2520000000000000000,0,0x6af958e749bd4b63131d28fce0125c3b1566afb668ee...,2625946,musd_btc_pool
4,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xd39abf814ba9bd8e7b64cb6505870b636d695d8f,0x52e604c44417233b6ccedddc0d640a405caacefb,0,21524636174687,2520000000000000000,0,0x12aa4abb6956d3eb42744f722f53772446374ff7dab2...,2625946,musd_btc_pool
...,...,...,...,...,...,...,...,...,...,...,...
32108,2025-07-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,250000000000000,0,0,28463880024065974068,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293,musd_btc_pool
32109,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x52e604c44417233b6ccedddc0d640a405caacefb,0,157198903110751,18910000000000000000,0,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335,musd_btc_pool
32110,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,2010400,0,0,2009989838895249558,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292,musd_musdc_pool
32111,2025-06-26,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x10906a9e9215939561597b4c8e4b98f93c02031a,0,999896001631977073,1000000,0,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305,musd_musdt_pool


In [16]:
pd.options.display.float_format = '{:.8f}'.format

In [17]:
from decimal import Decimal

def format_token_columns(df, cols, asset):
    def convert(x, token):
        if pd.isnull(x):
            return 0
        if token in {"USDC", "USDT"}:
            scale = Decimal("1e6")
        elif token in {"WBTC", "FBTC", "cbBTC", "swBTC"}:
            scale = Decimal("1e8")
        else:
            scale = Decimal("1e18")
        return float((Decimal(x) / scale).normalize())

    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df.apply(lambda row: convert(row[col], row[asset]), axis=1)
    
    return df

In [18]:
TOKEN_MAP = {
    '0x52e604c44417233b6ccedddc0d640a405caacefb': 'BTC',
    '0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356': 'USDC',
    '0x10906a9e9215939561597b4c8e4b98f93c02031a': 'USDT'
}

In [19]:
swaps_df['token'] = swaps_df['contractId_'].map(TOKEN_MAP)

In [21]:
format_token_columns(swaps_df, cols=['amount0In', 'amount0Out'], asset='token')

,timestamp_,sender,to,contractId_,amount0In,amount0Out,amount1In,amount1Out,transactionHash_,block_number,pool,token
0,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00001700,1990000000000000000,0,0x5337d050a1082d36701ecf7a7ceaf108f37a86a67432...,2625985,musd_btc_pool,BTC
1,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x1ddd75b7cd221cca5b39f4dd88c2d54737357361,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00004236,4960000000000000000,0,0xb5ca78eec6d0792d7dc74332d7ff9f9cfd8e0014302a...,2625970,musd_btc_pool,BTC
2,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000858,0.00000000,0,999000331300290689,0xcfdd976fbfc24f9c8a3ec978ae24d188ebfd75d2f6d5...,2625954,musd_btc_pool,BTC
3,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x99b340d3ee7b73de63f12f8b6ee667ea2d055ccd,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00002152,2520000000000000000,0,0x6af958e749bd4b63131d28fce0125c3b1566afb668ee...,2625946,musd_btc_pool,BTC
4,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xd39abf814ba9bd8e7b64cb6505870b636d695d8f,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00002152,2520000000000000000,0,0x12aa4abb6956d3eb42744f722f53772446374ff7dab2...,2625946,musd_btc_pool,BTC
...,...,...,...,...,...,...,...,...,...,...,...,...
32108,2025-07-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00025000,0.00000000,0,28463880024065974068,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293,musd_btc_pool,BTC
32109,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00015720,18910000000000000000,0,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335,musd_btc_pool,BTC
32110,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,2.01040000,0.00000000,0,2009989838895249558,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292,musd_musdc_pool,USDC
32111,2025-06-26,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x10906a9e9215939561597b4c8e4b98f93c02031a,0.00000000,999896001631.97705078,1000000,0,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305,musd_musdt_pool,USDT


In [22]:
format_musd_currency_columns(swaps_df, cols=['amount1In', 'amount1Out'])

,timestamp_,sender,to,contractId_,amount0In,amount0Out,amount1In,amount1Out,transactionHash_,block_number,pool,token
0,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00001700,1.99000000,0.00000000,0x5337d050a1082d36701ecf7a7ceaf108f37a86a67432...,2625985,musd_btc_pool,BTC
1,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x1ddd75b7cd221cca5b39f4dd88c2d54737357361,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00004236,4.96000000,0.00000000,0xb5ca78eec6d0792d7dc74332d7ff9f9cfd8e0014302a...,2625970,musd_btc_pool,BTC
2,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x7832453845e1e472af9a896ee348e628f7b36b81,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000858,0.00000000,0.00000000,0.99900033,0xcfdd976fbfc24f9c8a3ec978ae24d188ebfd75d2f6d5...,2625954,musd_btc_pool,BTC
3,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x99b340d3ee7b73de63f12f8b6ee667ea2d055ccd,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00002152,2.52000000,0.00000000,0x6af958e749bd4b63131d28fce0125c3b1566afb668ee...,2625946,musd_btc_pool,BTC
4,2025-08-22,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xd39abf814ba9bd8e7b64cb6505870b636d695d8f,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00002152,2.52000000,0.00000000,0x12aa4abb6956d3eb42744f722f53772446374ff7dab2...,2625946,musd_btc_pool,BTC
...,...,...,...,...,...,...,...,...,...,...,...,...
32108,2025-07-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x52d033e64774f31a8b7562806aa5782cbbd29382,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00025000,0.00000000,0.00000000,28.46388002,0x635f6d5a45a9bee6cdd11b40987141a6ca4225b7ab97...,1555293,musd_btc_pool,BTC
32109,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x52e604c44417233b6ccedddc0d640a405caacefb,0.00000000,0.00015720,18.91000000,0.00000000,0x32011a0f2a46f7c3560a3e7c7a341c3ecdad2d784cc8...,1530335,musd_btc_pool,BTC
32110,2025-07-07,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,2.01040000,0.00000000,0.00000000,2.00998984,0x3afbf42ee8974285284f7bc0a5f621a5022cee2f0a7e...,1530292,musd_musdc_pool,USDC
32111,2025-06-26,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80164ea60673d64d5d6228beb684a1274bb017,0x10906a9e9215939561597b4c8e4b98f93c02031a,0.00000000,999896001631.97705078,0.00000000,0.00000000,0x14869a1d4d2fd0a499d95c70d104132e1d6bc31372c7...,1264305,musd_musdt_pool,USDT


In [24]:
swaps_df['count'] = 1

In [26]:
GET_FEES_FOR_SWAPS = """
query swapFees($skip: Int!) {
  fees_collection(
  first: 1000
  orderBy: timestamp
  orderDirection: desc
  skip: $skip
  ) {
    timestamp
    txHash
    sender
    amount0
    amount1
    id
    pool {
      name
      symbol
      token0 {
        symbol
      }
      token1 {
        symbol
      }
    }
  }
}
"""

In [27]:
fees =  swaps.fetch_subgraph_data(GET_FEES_FOR_SWAPS, 'fees_collection')

Fetching transactions with skip=0...
No more records found.


In [ ]:
# daily
daily_swaps = swaps_df.groupby(['timestamp_']).agg(
    total_swaps = ('count', 'sum'),
    users = ('to', lambda x: x.nunique()),
    first_asset_in = ('amount0In', 'sum'),
    first_asset_out = ('amount0Out', 'sum'),
    musd_in = ('amount1In', 'sum'),
    musd_out = ('amount1Out', 'sum')
).reset_index()

In [ ]:
# daily
daily_swaps_by_pool = swaps_df.groupby(['timestamp_', 'pool']).agg(
    total_swaps = ('count', 'sum'),
    users = ('to', lambda x: x.nunique()),
    first_asset_in = ('amount0In', 'sum'),
    first_asset_out = ('amount0Out', 'sum'),
    musd_in = ('amount1In', 'sum'),
    musd_out = ('amount1Out', 'sum')
).reset_index()

In [ ]:
daily_swaps_by_pool = daily_swaps_by_pool.pivot(
    index='timestamp_', columns='pool'
).fillna(0)

In [ ]:
daily_swaps_by_pool.columns = [
'_'.join(col).strip() for col in daily_swaps_by_pool.columns.values
]

daily_swaps_by_pool = daily_swaps_by_pool.reset_index()

In [ ]:
# Next, convert to USD for cumulative, pct change, and rolling averages

In [ ]:
from mezo.currency_config import TOKENS_ID_MAP
from mezo.currency_utils import get_token_prices

def fetch_token_prices():
    prices = get_token_prices()
    if prices is None or prices.empty:
        raise ValueError("No token prices received from API")
    return prices

tokens = fetch_token_prices()
token_usd_prices = tokens.T.reset_index()

swaps_df['index'] = swaps_df['token'].map(TOKENS_ID_MAP)

In [ ]:
swaps_df

In [ ]:
swaps_df_with_usd = pd.merge(swaps_df, token_usd_prices, how='left', on='index')
swaps_df_with_usd['amount_usd_in'] = swaps_df_with_usd['amount0In'] * swaps_df_with_usd['usd']
swaps_df_with_usd['amount_usd_out'] = swaps_df_with_usd['amount0Out'] * swaps_df_with_usd['usd']

In [ ]:
swaps_df_with_usd

In [ ]:
swaps_df_with_usd.dtypes

In [ ]:
# daily
daily_swaps_usd = swaps_df_with_usd.groupby(['timestamp_']).agg(
    total_swaps = ('count', 'sum'),
    users = ('to', lambda x: x.nunique()),
    first_asset_in = ('amount0In', 'sum'),
    first_asset_in_usd = ('amount_usd_in', 'sum'),
    first_asset_out = ('amount0Out', 'sum'),
    first_asset_out_musd = ('amount_usd_out', 'sum'),
    musd_in = ('amount1In', 'sum'),
    musd_out = ('amount1Out', 'sum')
).reset_index()

In [ ]:
daily_swaps_usd['volume'] = (
        daily_swaps_usd['musd_in'] + 
        daily_swaps_usd['musd_out']
    )

In [ ]:
daily_swaps_usd['volume']

In [ ]:
add_rolling_values(daily_swaps_usd, 7, ['volume'])

In [ ]:
daily_swaps_usd = add_rolling_values(daily_swaps_usd, 30, ['volume'])

In [ ]:
daily_swaps_usd

In [ ]:
daily_swaps_by_pool_usd = swaps_df_with_usd.groupby(['timestamp_', 'pool']).agg(
    total_swaps = ('count', 'sum'),
    users = ('to', lambda x: x.nunique()),
    first_asset_in = ('amount0In', 'sum'),
    first_asset_in_usd = ('amount_usd_in', 'sum'),
    first_asset_out = ('amount0Out', 'sum'),
    first_asset_out_usd = ('amount_usd_out', 'sum'),
    musd_in = ('amount1In', 'sum'),
    musd_out = ('amount1Out', 'sum')
).reset_index()

In [ ]:
daily_swaps_by_pool_usd

In [ ]:
daily_swaps_by_pool_usd = daily_swaps_by_pool_usd.pivot(
    index='timestamp_', columns='pool'
).fillna(0)

In [ ]:
daily_swaps_by_pool_usd.columns = [
'_'.join(col).strip() for col in daily_swaps_by_pool_usd.columns.values
]

daily_swaps_by_pool_usd = daily_swaps_by_pool_usd.reset_index()

In [ ]:
daily_swaps_by_pool_usd

In [ ]:
def add_pool_volume_columns(df, in_suffix='_in', out_suffix='_out'):
    """
    Add volume columns for each pool in a pivoted daily swaps dataframe.
    
    Args:
        df: DataFrame with columns like 'musd_in_POOL1', 'musd_out_POOL1', etc.
        in_suffix: Suffix for inflow columns (default: '_in')
        out_suffix: Suffix for outflow columns (default: '_out')
    
    Returns:
        DataFrame with additional 'volume_POOL' columns
    """
    # Extract pool names from column names in one pass
    in_pools = {col.replace(f'musd{in_suffix}_', '') for col in df.columns if col.startswith(f'musd{in_suffix}_')}
    out_pools = {col.replace(f'musd{out_suffix}_', '') for col in df.columns if col.startswith(f'musd{out_suffix}_')}
    
    # Only create volume columns for pools that have both in and out columns
    pools_with_both = in_pools & out_pools
    
    # Create volume columns using dictionary comprehension
    volume_columns = {
        f'volume_{pool}': df[f'musd{in_suffix}_{pool}'] + df[f'musd{out_suffix}_{pool}']
        for pool in pools_with_both
    }
    
    return df.assign(**volume_columns)


In [ ]:
daily_swaps_by_pool_usd_with_vol = add_pool_volume_columns(
    daily_swaps_by_pool_usd, 
    in_suffix='_in', 
    out_suffix='_out'
)

In [ ]:
daily_swaps_by_pool_usd_with_vol['volume_musd_btc_pool',
       'volume_musd_musdc_pool', 'volume_musd_musdt_pool']

In [ ]:

from mezo.clients import Web3Client

w3 = Web3Client('PoolRouter')
router = w3.load_contract()
router.all_functions()